# Проект: Анализ вакансий из HeadHunter

Изучаем данные о вакансиях с сайта HeadHunter.

In [1]:
#!pip install psycopg2

In [2]:
#Импортируем нужные библиотеки
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup

In [3]:
#Данные для подключения к базе 
DBNAME = '*************'
USER = '*************'
PASSWORD = '*************'
HOST = '*************'
PORT = ****

In [4]:
#Создаём подключение
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [5]:
# текст запроса
query_3_1 = f'''
            SELECT COUNT(*) AS vacancies_count
            FROM public.vacancies
'''

In [6]:
# используем метод read_sql_query, чтобы получить результаты запроса сразу в виде датафрейма
vac_count = pd.read_sql_query(query_3_1, connection)
print(f'Количество вакансий в базе: {vac_count.vacancies_count[0]}')

Количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [7]:
# текст запроса
query_3_2 = f'''
            SELECT COUNT(*) AS employers_count
            FROM public.employers
'''

In [8]:
# результат запроса
emp_count = pd.read_sql_query(query_3_2, connection)
print(f'Количество работодателей: {emp_count.employers_count[0]}')

Количество работодателей: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [9]:
# текст запроса
query_3_3 = f'''
            SELECT COUNT(*) AS areas_count
            FROM public.areas
'''

In [10]:
# результат запроса
areas_count = pd.read_sql_query(query_3_3, connection)
print(f'Количество регионов: {areas_count.areas_count[0]}')

Количество регионов: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [11]:
# текст запроса
query_3_4 = f'''
            SELECT COUNT(*) AS industries_count
            FROM public.industries
'''

In [12]:
# результат запроса
ind_count = pd.read_sql_query(query_3_4, connection)
print(f'Количество сфер деятельности: {ind_count.industries_count[0]}')

Количество сфер деятельности: 294


***

## Выводы по предварительному анализу данных

- Данные включают 5 таблиц: vacancies, employers, areas, employers_industries, industries.

- Общее количество вакансий в таблице vacancies - 49197

- Общее количество работотаделей в таблице employers - 23501

- Количество регионов, представленных в таблице areas - 1362

- Количество сфер деятельности, представленных в таблице industries - 294

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

Вывести пятёрку регионов-лидеров по количествю вакансий.

In [13]:
# текст запроса
query_4_1 = f'''
            SELECT COUNT(vac.name) AS vacancy_count, 
                   ar.name AS region_name
            FROM public.vacancies AS vac
            JOIN public.areas AS ar
            ON vac.area_id=ar.id
            GROUP BY region_name
            ORDER BY vacancy_count desc
            LIMIT 5
'''

In [14]:
# результат запроса
vac_by_areaid_count = pd.read_sql_query(query_4_1, connection)
vac_by_areaid_count


,vacancy_count,region_name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [15]:
# текст запроса
query_4_2 = f'''
            SELECT COUNT(id) AS vacancy_count
            FROM public.vacancies
            WHERE salary_from IS NOT Null OR salary_to IS NOT Null
'''

In [16]:
# результат запроса
not_null_salary_count = pd.read_sql_query(query_4_2, connection)
print(f'Количество вакансий с заполненными полями с зарплатой: {not_null_salary_count.vacancy_count[0]}')

Количество вакансий с заполненными полями с зарплатой: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [17]:
# текст запроса
query_4_3 = f'''
            SELECT ROUND(AVG(salary_from), 0) AS avg_min_salary, 
                   ROUND(AVG(salary_to), 0) AS avg_max_salary
            FROM public.vacancies
'''

In [18]:
# результат запроса
avg_salaryes = pd.read_sql_query(query_4_3, connection)
avg_salaryes
print(f'Среднее значение для нижней границы зарплатной вилки: {avg_salaryes.avg_min_salary[0]}')
print(f'Среднее значение для верхней границы зарплатной вилки: {avg_salaryes.avg_max_salary[0]}')

Среднее значение для нижней границы зарплатной вилки: 71065.0
Среднее значение для верхней границы зарплатной вилки: 110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [19]:
# текст запроса
query_4_4 = f'''
            SELECT COUNT(id) AS vacancy_count, 
                   schedule, 
                   employment
            FROM public.vacancies
            GROUP BY schedule, employment
            ORDER BY vacancy_count DESC
            LIMIT 5
'''

In [20]:
# результат запроса
vacancy_count = pd.read_sql_query(query_4_4, connection)
vacancy_count

,vacancy_count,schedule,employment
0,35367,Полный день,Полная занятость
1,7802,Удаленная работа,Полная занятость
2,1593,Гибкий график,Полная занятость
3,1312,Удаленная работа,Частичная занятость
4,940,Сменный график,Полная занятость


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [21]:
# текст запроса
query_4_5 = f'''
            SELECT experience, 
                   COUNT(name) AS vacancy_count
            FROM public.vacancies
            GROUP BY experience
            ORDER BY vacancy_count ASC
'''

In [22]:
# результат запроса
experience_sorted = pd.read_sql_query(query_4_5, connection)
experience_sorted


,experience,vacancy_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## Выводы по детальному анализу вакансий

- Больше всего вакансий предлагается в таких городах, как: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы. Причём, в Москве вакансий в два раза больше, чем в ближайшем к ней по числу вакансий Санкт-Петербурге.

- Данные о зарплате заполнены примерно у половины всех вакансий. 

- Средние значения нижней и верхней границ по зарплатам: 71065 и 110537

- Наиболее популярные сочетания типа рабочего графика и типа трудоустройства следующие: Полный день - Полная занятость, Удалённая работа - Полная занятость.

- Требования к опыту во всей совокупности изучаемых вакансий распределились так: больше всего вакансий требуют опыт от 1 до 3 лет, в два раза меньше вакансий с требованием к опыту от 3 до 6 лет, ещё в два раза меньше вакансий для соискателей без опыта работы, и лишь около 2.5% всех вакансий для соискателей с опытом более 6 лет.


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [23]:
# текст запроса
query_5_1 = f'''
            SELECT emp.name AS emp_name,
                   COUNT(vac.name) AS vac_count
            FROM public.vacancies AS vac
            JOIN public.employers AS emp
            ON vac.employer_id=emp.id
            GROUP BY emp_name
            ORDER BY vac_count DESC
            LIMIT 5
'''

In [24]:
# результат запроса
emp_count_vac = pd.read_sql_query(query_5_1, connection)
emp_count_vac

,emp_name,vac_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [25]:

query_5_2 = f'''
            SELECT ar.name AS region,
                   COUNT(DISTINCT(emp.name)) AS employer_count,
                   COUNT(DISTINCT(vac.name)) AS vacancies_count
            FROM public.areas AS ar
            LEFT JOIN public.vacancies AS vac
            ON ar.id=vac.area_id
            LEFT JOIN public.employers AS emp
            ON emp.area=ar.id
            WHERE vac.name IS Null
            
            GROUP BY region
            ORDER BY employer_count DESC
            LIMIT 5
                       
'''

In [26]:
# результат запроса
area_emp_vac_count = pd.read_sql_query(query_5_2, connection)
area_emp_vac_count

,region,employer_count,vacancies_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [27]:
query_5_3 = f'''
            SELECT emp.name AS emp_name,
                   COUNT(DISTINCT(vac.area_id)) AS area_count
            
            FROM public.vacancies AS vac
            LEFT JOIN public.employers AS emp
            ON vac.employer_id=emp.id
            GROUP BY emp.name
            ORDER BY area_count DESC
            LIMIT 5
'''

In [28]:
# результат запроса
employers = pd.read_sql_query(query_5_3, connection)
employers

,emp_name,area_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 


In [29]:
#Для этого сравним две таблицы: 
#- в таблице employers_industries может не быть записей о работодателях из таблицы employers.
#- поэтому объединим их с помощью left join (employers - левая таблица) и найдём строки, в которых industry_id имеет значение Null
#- посчитаем количество строк, соответствующих этому условию 

query_5_4 = f'''
            SELECT COUNT(*) AS emp_count
            FROM public.employers AS emp
            LEFT JOIN public.employers_industries AS emp_ind
            ON emp.id=emp_ind.employer_id
            WHERE industry_id IS Null
'''

In [30]:
# результат запроса
ind = pd.read_sql_query(query_5_4, connection)
ind
print(f'Число работодателей с неуказанной сферой деятельности: {ind.emp_count[0]}')

Число работодателей с неуказанной сферой деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [31]:
query_5_5 = f'''
            SELECT name,
                   COUNT(industry_id) AS ind_count
            FROM public.employers AS emp
            LEFT JOIN public.employers_industries AS emp_ind
            ON emp.id=emp_ind.employer_id
            GROUP BY name
            HAVING COUNT(industry_id) = 4
            ORDER BY name
            LIMIT 3
'''

In [32]:
# результат запроса
ind_4 = pd.read_sql_query(query_5_5, connection)
display(ind_4)
print()
print(f'Название третьей компании: {ind_4.name[2]}')


,name,ind_count
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4



Название третьей компании: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [33]:
query_5_6 = f'''
            SELECT COUNT(*) developers_count
            FROM public.employers AS emp
            LEFT JOIN public.employers_industries AS emp_ind
            ON emp.id=emp_ind.employer_id
            LEFT JOIN public.industries AS ind
            ON emp_ind.industry_id=ind.id
            WHERE ind.name='Разработка программного обеспечения'
'''

In [34]:
# результат запроса
developers = pd.read_sql_query(query_5_6, connection)
print(f'Количество работодателей в области "Разработка программного обеспечения": {developers.developers_count[0]}')

Количество работодателей в области "Разработка программного обеспечения": 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 


In [35]:
#Получим кортеж с перечнем городов-миллионников
URL = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
response = requests.get(URL, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'})
page = BeautifulSoup(response.text, 'html.parser') 
links = page.find_all('a')
cities = tuple([link.text for link in links[915:931]]) #переведём в кортеж для обработки в SQL запросе
print(cities)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [36]:
# текст запроса
#cities = ('Москва','Санкт-Петербург','Новосибирск','Екатеринбург','Казань','Нижний Новгород',
#          'Челябинск','Красноярск','Самара','Уфа','Ростов-на-Дону','Омск','Краснодар','Воронеж','Пермь','Волгоград')

query_16 = f'''
            WITH emp_cities AS 
            (SELECT ar.name AS city,
                    COUNT(*) AS vacancy_count
            
            FROM public.vacancies AS vac
            LEFT JOIN public.employers AS emp
            ON emp.id=vac.employer_id
            LEFT JOIN public.areas AS ar
            ON ar.id=vac.area_id
            
            WHERE emp.name='Яндекс' AND ar.name IN {cities}
            
            GROUP BY ar.name
            ORDER BY vacancy_count ASC)
            
            --Создаём итоговую таблицу с суммой по колонке--
            
            SELECT * from emp_cities
            UNION ALL
            SELECT 'Total',
                    SUM(vacancy_count) 
            FROM emp_cities
'''

In [37]:
# результат запроса
cit = pd.read_sql_query(query_16, connection)
cit

,city,vacancy_count
0,Омск,21.0
1,Челябинск,22.0
2,Красноярск,23.0
3,Волгоград,24.0
4,Пермь,25.0
5,Казань,25.0
6,Ростов-на-Дону,25.0
7,Уфа,26.0
8,Самара,26.0
9,Краснодар,30.0


***

## Выводы по анализу работодателей

- Больше всего вакансий предлагают такие крупные компании как: Яндекс (лидирует с большим отрывом), Ростелеком, Тинькофф, СБЕР и Газпром нефть.

- Четверть всех вакансий компании Яндекс (485 вакансий) представлена в городах-миллионниках. Больше всего вакансий компании открыто в Москве и Санкт-Петербурге. Меньше всего вакансий компании в Омске, Челябинске и Красноярске.

- Вакансии по разработке программного обеспечения предлагает 3553 работодателя.

- Крупные компании, типа Яндекса, предлагают вакансии в большом количестве регионов (Яндекс в 181 регионе).

- Россия и Казахстан лидируют по числу компаний, в которых на данный момент нет открытых вакансий. 410 и 207 компаний соответственно.


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [38]:
# текст запроса
query_6_1 = f'''
                SELECT name
                FROM public.vacancies
                WHERE lower(name) LIKE '%data%' OR lower(name) LIKE '%данн%'              
'''

In [39]:
# результат запроса
vac_data = pd.read_sql_query(query_6_1, connection)
print('Пример отфильтрованных вакансий:')
display(vac_data.head(5))
print()
print(f'Количество вакансий, связанных с данными: {vac_data.shape[0]}')

Пример отфильтрованных вакансий:


,name
0,Аналитик хранилищ данных и BI-систем/Аналитик ...
1,Data-analysts (удаленно)
2,Стажер Data Engineer/Data Аналитик в команду Б...
3,Старший специалист хранилищ данных
4,Администратор баз данных и сетей



Количество вакансий, связанных с данными: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [40]:
# текст запроса
#Сделана максимально детализированная выборка, с ответом на платформе не совпадает.
query_6_2 = f'''
                WITH data_vacs AS(
                SELECT * 
                FROM public.vacancies
                WHERE 
                     lower(name) LIKE '%data scientist%' OR
                     lower(name) LIKE '%исследователь данных%' OR 
                     lower(name) SIMILAR TO '%[/| ]ml%' OR 
                     lower(name) SIMILAR TO '%[(]ml%' OR
                     lower(name) LIKE 'ml%' OR 
                     lower(name) LIKE '%machine learning%' OR
                     lower(name) LIKE '%машинн%обучен%'
                )
                
                SELECT *
                FROM data_vacs
                WHERE
                     lower(name) LIKE '%junior%' OR
                     experience = 'Нет опыта' OR
                     employment = 'Стажировка'
'''

In [41]:
# результат запроса
data_vacs = pd.read_sql_query(query_6_2, connection)
print(f'Количество вакансий, связанных с данными для джуниоров: {data_vacs.shape[0]}')


Количество вакансий, связанных с данными для джуниоров: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [42]:
# текст запроса
query_6_3 = f'''
                WITH data_vacs AS(
                SELECT * 
                FROM public.vacancies
                WHERE 
                     lower(name) LIKE '%data scientist%' OR
                     lower(name) LIKE '%исследователь данных%' OR 
                     lower(name) SIMILAR TO '%[/| ]ml%' OR 
                     lower(name) SIMILAR TO '%[(]ml%' OR
                     lower(name) LIKE 'ml%' OR 
                     lower(name) LIKE '%machine learning%' OR
                     lower(name) LIKE '%машинн%обучен%'
                )
                
                SELECT *
                FROM data_vacs
                WHERE
                     lower(key_skills) LIKE '%sql%' OR
                     lower(key_skills) LIKE '%postgres%'
'''


In [43]:
# результат запроса
sql_ds_vacs = pd.read_sql_query(query_6_3, connection)
print(f'Количество вакансий, связанных с SQL и postgres: {sql_ds_vacs.shape[0]}')


Количество вакансий, связанных с SQL и postgres: 182


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.

Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [44]:
# текст запроса
query_6_4 = f'''
                WITH data_vacs AS(
                SELECT * 
                FROM public.vacancies
                WHERE 
                     lower(name) LIKE '%data scientist%' OR
                     lower(name) LIKE '%исследователь данных%' OR 
                     lower(name) SIMILAR TO '%[/| ]ml%' OR 
                     lower(name) SIMILAR TO '%[(]ml%' OR
                     lower(name) LIKE 'ml%' OR 
                     lower(name) LIKE '%machine learning%' OR
                     lower(name) LIKE '%машинн%обучен%'
                )
                
                SELECT *
                FROM data_vacs
                WHERE
                     lower(key_skills) LIKE '%python%'
'''

In [45]:
# результат запроса
python_ds_vacs = pd.read_sql_query(query_6_4, connection)
print(f'Количество вакансий, связанных с Python: {python_ds_vacs.shape[0]}')


Количество вакансий, связанных с Python: 325


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?

Ответ округлите до двух знаков после точки-разделителя.
указаны в тексте вакансии.

In [46]:
#В решении использована идея о замене символа табуляции '\t' пустым значением.
#После этого находится разница между длиной полной строки и длиной строки с убранными знаками табуляции.
#Таким образом получаем количество навыков минус 1. 

query_6_5 = f'''
                WITH data_vacs AS(
                SELECT * 
                FROM public.vacancies
                WHERE 
                     lower(name) LIKE '%data scientist%' OR
                     lower(name) LIKE '%исследователь данных%' OR 
                     lower(name) SIMILAR TO '%[/| ]ml%' OR 
                     lower(name) SIMILAR TO '%[(]ml%' OR
                     lower(name) LIKE 'ml%' OR 
                     lower(name) LIKE '%machine learning%' OR
                     lower(name) LIKE '%машинн%обучен%'
                )
                
                SELECT AVG(LENGTH(CAST(key_skills AS CHAR(255))) - LENGTH(CAST(REPLACE(key_skills, '\t', '') AS CHAR(255))) + 1) AS skills_average_number
                FROM data_vacs
                WHERE LENGTH(CAST(key_skills AS CHAR(255))) - LENGTH(CAST(REPLACE(key_skills, '\t', '') AS CHAR(255))) > 0

'''

In [47]:
# результат запроса
skills_ds_vacs = pd.read_sql_query(query_6_5, connection)
print(f'Среднее число навыков в вакансиях Data Science: {round(skills_ds_vacs.skills_average_number[0], 2)}')

Среднее число навыков в вакансиях Data Science: 6.32


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [48]:
query_6_6 = f'''
                WITH data_vacs AS(
                SELECT * 
                FROM public.vacancies
                WHERE 
                     lower(name) LIKE '%data scientist%' OR
                     lower(name) LIKE '%исследователь данных%' OR 
                     lower(name) SIMILAR TO '%[/| ]ml%' OR 
                     lower(name) SIMILAR TO '%[(]ml%' OR
                     lower(name) LIKE 'ml%' OR 
                     lower(name) LIKE '%machine learning%' OR
                     lower(name) LIKE '%машинн%обучен%'
                ),
                
                average_salary AS(
                SELECT experience,
                       salary_from,
                       salary_to,
                       CASE WHEN salary_from IS NOT Null AND salary_to IS NOT Null THEN (salary_from + salary_to)/2
                            WHEN salary_from IS Null AND salary_to IS NOT Null THEN salary_to
                            WHEN salary_from IS NOT Null AND salary_to IS Null THEN salary_from 
                       END AS salary
                FROM data_vacs
                WHERE salary_from IS NOT Null OR salary_to IS NOT Null
                )
                
                SELECT experience,
                       AVG(salary) AS avg_salary
                FROM average_salary
                GROUP BY experience
'''

In [49]:
# результат запроса
sal = pd.read_sql_query(query_6_6, connection)
print(f'Средняя зарплата для датасайентиста с опытом работы от 3 до 6 лет: {round(sal.avg_salary[0])}')


Средняя зарплата для датасайентиста с опытом работы от 3 до 6 лет: 240927


***

## Выводы по предметному анализу

- Общее количество вакансий, связанных с данными: 1771 (из них только 426 напрямую относятся к Data Science).

- Количество вакансий в области Data Science для джуниоров: 51 из 426.

- Количество вакансий в области Data Science, связанных с SQL и postgres: 182 из 426.

- Количество вакансий в области Data Science, связанных с Python: 325 из 426.

- Среднее число навыков в вакансиях в области Data Science: 6.32. Интересно изучить, какие это навыки.

- Средняя зарплата для датасайентиста с опытом работы от 3 до 6 лет составляет 240927 рублей в месяц.

# Общий вывод по проекту

## Общие сведения

Данные включают 5 таблиц: vacancies, employers, areas, employers_industries, industries.

Общее количество вакансий в таблице vacancies - 49197

Общее количество работотаделей в таблице employers - 23501

Количество регионов, представленных в таблице areas - 1362

Количество сфер деятельности, представленных в таблице industries - 294


## Работодатели

Больше всего вакансий предлагают такие крупные компании как: Яндекс (лидирует с большим отрывом), Ростелеком, Тинькофф, СБЕР и Газпром нефть.

Четверть всех вакансий компании Яндекс (485 вакансий) представлена в городах-миллионниках. Больше всего вакансий компании открыто в Москве и Санкт-Петербурге. Меньше всего вакансий компании в Омске, Челябинске и Красноярске.

- компания Яндекс очень активно ищет сотрудников в провинции

Вакансии по разработке программного обеспечения предлагает 3553 работодателя.

- только 15% работодателей относятся к сфере разработки софта

Крупные компании, типа Яндекса, предлагают вакансии в большом количестве регионов (Яндекс в 181 регионе).

Россия и Казахстан лидируют по числу компаний, в которых на данный момент нет открытых вакансий. 410 и 207 компаний соответственно.

- вполне возможно, по этим данным можно косвенно судить о насыщении рынка вакансий в определённых сферах деятельности


## Вакансии

Больше всего вакансий предлагается в таких городах, как: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы. Причём, в Москве вакансий в два раза больше, чем в ближайшем к ней по числу вакансий Санкт-Петербурге. (Вполне логичная картина).

Данные о зарплате заполнены примерно у половины всех вакансий.

- по этой причине судить о зарплатах в некоторых сферах деятельности достаточно проблематично. Нужны дополнительные источники информации.

Средние значения нижней и верхней границ по зарплатам: 71065 и 110537 

- интересно сравнить зарплаты в области Data Science с общей статистикой по зарплатам.

Наиболее популярные сочетания типа рабочего графика и типа трудоустройства следующие: Полный день - Полная занятость, Удалённая работа - Полная занятость.

Требования к опыту во всей совокупности изучаемых вакансий распределились так: 

- в большинстве вакансий требуется опыт от 1 до 3 лет (проще всего найти работу людям с опытом от 1 года)
- в два раза меньше вакансий с требованием к опыту от 3 до 6 лет
- ещё в два раза меньше вакансий для соискателей без опыта работы (найти работу без опыта достаточно трудно) 
- и лишь около 2.5% всех вакансий для соискателей с опытом более 6 лет

## Вакансии в области Data Science

Общее количество вакансий, связанных с данными: 1771 из почти 50000 (из них только 426 напрямую относятся к Data Science).

- только 3.5% вакансий связаны с работой с данными
- и лишь около 1% вакансий относятся к области Data Science

Количество вакансий в области Data Science для джуниоров: 51 из 426.

- 12% вакансий в области Data Science - для начинающих специалистов.

Количество вакансий в области Data Science, связанных с SQL и postgres: 182 из 426.

- 43% вакансий в области Data Science требуют знания SQL или PostgreSQL.

Количество вакансий в области Data Science, связанных с Python: 325 из 426.

- 76% вакансий в области Data Science требуют знания языка программирования Python.

Среднее число навыков в вакансиях в области Data Science: 6.32. Интересно изучить, какие это навыки.

- Вакансии в области Data Science требуют довольно много навыков - более 6 в среднем. Какие это навыки - требует отдельного изучения.

Средняя зарплата для датасайентиста с опытом работы от 3 до 6 лет составляет 240927 рублей в месяц.

## Дополнительные исследования

Имеет смысл провести дополнительные исследования для ответа на вопросы:

- В каких регионах больше всего компаний, разрабатывающих программное обеспечение?

- Какие компании охватывают максимальное количество сфер деятельности? 

- Какие навыки в области Data Science наиболее востребованы?

- Какой набор навыков ценится наиболее высоко в области Data Science?

- В каких регионах больше всего вакансий в области работы с данными?

- В каких регионах самые высокие зарплаты у специалистов по данным? Интересно сравнить зарплаты в области Data Science с общей статистикой по зарплатам.

- Отдельного изучения требует распределение разных вакансий во времени. К сожалению, наши данные не дают такой информации. 